Setup the spark session. These files are small, so we can fit more tasks onto executors. Seems like 4 cores / 4GB works. Total of 120 cores means we can fit 29 (since we need a core for the driver)

In [12]:
%%configure -f
{"name": "arik-load-logcube-write", "executorMemory": "4G", "numExecutors": 29, "executorCores": 4,
 "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"python3"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1588740809550_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1588740809550_0006,pyspark,idle,Link,Link,✔


In [18]:
import sys
import os
import subprocess
from io import BytesIO
from gzip import GzipFile
from pyspark.sql import Row
import glob
import time
from itertools import repeat

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
def get_fits_module():
    if 'astropy' not in sys.modules:
        stdout = subprocess.check_output(
            sys.executable + ' -m pip install astropy',
            stderr=subprocess.STDOUT,
            shell=True).decode('utf-8')
    from astropy.io import fits
    return fits

def get_wcs_module():
    if 'astropy' not in sys.modules:
        stdout = subprocess.check_output(
            sys.executable + ' -m pip install astropy',
            stderr=subprocess.STDOUT,
            shell=True).decode('utf-8')
    from astropy import wcs
    return wcs

def getfitslocal(path):
    fits = get_fits_module()
    fits_obj = fits.open(path)
    return fits_obj

# Spark dataframe cannot do numpy types
def typeconv(i):
    try:
        return i.item()
    except:
        return i

def createSpaxelMapRows(fits):
    # Scan through X, Y pixel space of flux cube, discard empty spectra
    rows = []
    n_y, n_x = fits['SPX_SNR'].data.shape
    # prepare data for EMLINE fields
    hdu = fits['EMLINE_SFLUX']
    emlines = [hdu.header['C{:02d}'.format(i+1)] for i in range(hdu.data.shape[0])]
    emline_hdus = [i.name for i in fits if i.name.startswith('EMLINE_')]
    # Stellar
    stellar_hdus = [i.name for i in fits if i.name.startswith('STELLAR_') and i.name!='STELLAR_CONT_FRESID']
    # flat image dimension hdus
    flat_hdus = ['SPX_MFLUX', 'SPX_MFLUX_IVAR', 'SPX_SNR', 
                 'BIN_AREA', 'BIN_FAREA', 'BIN_MFLUX', 'BIN_MFLUX_IVAR', 'BIN_MFLUX_MASK', 'BIN_SNR'] + \
        stellar_hdus
    # prepare coordinate info
    objra, objdec = typeconv(fits[0].header['OBJRA']), typeconv(fits[0].header['OBJDEC'])
    # these headers are specific to file, so collect them in appropriate format before loop
    per_file_data = [(i, typeconv(fits[0].header[i])) for i in ['PLATEID', 'IFUDSGN', 'MJDMED']]
    for x in range(n_x):
        for y in range(n_y):
            # Skip if the spaxel is not populated, using g-band mean flux as proxy
            if fits['SPX_MFLUX'].data[y, x] == 0:
                continue
            base = [(hdu, typeconv(fits[hdu].data[y, x])) for hdu in flat_hdus] + [ \
                ('RA', objra + typeconv(fits['SPX_SKYCOO'].data[0, y, x])),
                ('DEC', objdec + typeconv(fits['SPX_SKYCOO'].data[1, y, x])),
                ('GAL_R', typeconv(fits['SPX_ELLCOO'].data[0, y, x])),
                ('GAL_R_REFF', typeconv(fits['SPX_ELLCOO'].data[1, y, x])),
                ('GAL_THETA', typeconv(fits['SPX_ELLCOO'].data[2, y, x])),
                ('STELLAR_CONT_FRESID_68', typeconv(fits['STELLAR_CONT_FRESID'].data[0, y, x])),
                ('STELLAR_CONT_FRESID_99', typeconv(fits['STELLAR_CONT_FRESID'].data[0, y, x])),
                ('BIN_LWSKYCOO_RA', typeconv(fits['BIN_LWSKYCOO'].data[0, y, x])),
                ('BIN_LWSKYCOO_DEC', typeconv(fits['BIN_LWSKYCOO'].data[1, y, x])),    
                ('BIN_LWELLCOO_GAL_R', typeconv(fits['BIN_LWELLCOO'].data[0, y, x])),
                ('BIN_LWELLCOO_GAL_R_REFF', typeconv(fits['BIN_LWELLCOO'].data[1, y, x])),
                ('BIN_LWELLCOO_GAL_THETA', typeconv(fits['BIN_LWELLCOO'].data[2, y, x])),
            ]
            for l in range(len(emlines)):
                row = [('LINE', emlines[l]), ('_SRC_X', x), ('_SRC_Y', y)] + \
                    base + \
                    per_file_data + \
                    [(hdu, typeconv(fits[hdu].data[l, y, x])) for hdu in emline_hdus]
                rows.append(Row(**dict(row)))
    return rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read local fits to parquet
Here we scan all directories that have a stack subdir and identify all LOGRSS files within them. Each file will be given it's own partition, so first we scan for the total dataset size including number of files, sum of filesize and max filesize (this will determine how much memory tasks need).

In [20]:
base_dir = '/sciserver/vc/manga/vc/sas/dr15/manga/spectro/analysis/v2_4_3/2.2.1/VOR10-GAU-MILESHC/'
dirs1 = [base_dir + i for i in os.listdir(base_dir)]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
files_rdd = sc.parallelize(dirs1, len(dirs1)).flatMap(lambda x: [x+'/'+i for i in os.listdir(x)]).flatMap(lambda d: glob.glob(d+'/*-MAPS-*.fits.gz'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
files_stats = files_rdd.map(
    lambda x: (os.stat(x).st_size/1024/1024, 1, os.stat(x).st_size/1024/1024)
).reduce(
    lambda x,y: (x[0]+y[0], x[1]+y[1], max(x[2],y[2]))
)
print('N files: {1}. Total Size: {0:0.0f}MB, average size: {2:0.0f}'.format(*files_stats))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

N files: 4722. Total Size: 3736MB, average size: 4

In [7]:
n_part = int(files_stats[1])
table_data = files_rdd.repartition(n_part).map(getfitslocal).flatMap(createSpaxelMapRows)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### WARNING
This is a time consuming job. The memory requirements are not so high (can fit in 1G per task), but the data conversion throughput is very low - so takes a while (16hrs / ~12sec per file on average). The dataset size remains about the same ~4G in fits and in this parquet format.

In [13]:
hdfs_dir = 'hdfs:///manga/arik-test/dr15/v2_4_3/maps'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
t = time.time()
table = spark.createDataFrame(table_data)
table.write.mode('overwrite').parquet(hdfs_dir)
print(time.time()-t)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

61066.266315698624

In [14]:
df = spark.read.parquet(hdfs_dir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df.createOrReplaceTempView('data')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
spark.sql('''select count(*) from data''').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
| count(1)|
+---------+
|182235416|
+---------+

In [19]:
t = time.time()
spark.sql('''
SELECT plateid, ifudsgn, count(*), min(_src_x), max(_src_x) FROM data GROUP BY plateid, ifudsgn
''').show()
print(time.time()-t)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+--------+-----------+-----------+
|plateid|ifudsgn|count(1)|min(_src_x)|max(_src_x)|
+-------+-------+--------+-----------+-----------+
|   8084|  12704|   68970|          5|         72|
|   8993|   3702|   19712|          4|         40|
|   9049|   3703|   19492|          3|         38|
|   8440|   6104|   32054|          5|         51|
|   8552|   9101|   47036|          6|         61|
|   9881|   3702|   20482|          4|         41|
|   8085|  12703|   65758|          5|         69|
|   8486|   3702|   20416|          5|         41|
|   8448|   1901|   10692|          5|         30|
|   8261|   3701|   19646|          5|         40|
|   8724|   9101|   47938|          4|         60|
|   8985|   3701|   19976|          5|         41|
|   7443|  12701|   61996|          4|         68|
|   8262|  12701|   65736|          6|         71|
|   8315|   3701|   19778|          6|         41|
|   7964|  12705|   68354|          5|         71|
|   9507|   9101|   47454|     

In [30]:
spark.sql('''SELECT count(*) FROM data WHERE EMLINE_SFLUX!=0''').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
| count(1)|
+---------+
|124228735|
+---------+

So some portion of lines have no flux (even when they have some g-band total flux). That portion is:

In [31]:
124228735/182235416*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

68.16936999776158